In [ ]:
import pandas as pd
from keras import Sequential
from keras.src.callbacks import ModelCheckpoint
from keras.src.layers import Dense
import plotly.graph_objects as go
import plotly.express as px
from keras.src.optimizers import Adam
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [ ]:
data = pd.read_csv("housing_price_dataset.csv")
labels = data["Price"]
data = data.drop("Price", axis=1)
data

In [ ]:
plt = px.histogram(labels)
plt

In [ ]:
data = pd.get_dummies(data, columns=["Neighborhood"], dtype=int)
data

In [ ]:
scaler = StandardScaler()
data = scaler.fit_transform(data)
data

In [ ]:
model = Sequential()
model.add(Dense(units=512, activation="relu", input_shape=(7,)))
model.add(Dense(units=12, activation="relu"))
model.add(Dense(units=1))

model.summary()

In [ ]:
mc = ModelCheckpoint(filepath="best_model3.keras", monitor="val_mae", verbose=1, mode="min", save_best_only=True)

model.compile(optimizer=Adam(learning_rate=0.01, beta_1=0.8, beta_2=0.95),
              loss="mse",
              metrics=["mae"])

history = model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2, callbacks=[mc])

model.save("model_after_training.keras")

In [ ]:
def draw_history(history):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history["mae"], name="mae"))
    fig.add_trace(go.Scatter(x=history.epoch, y=history.history["val_mae"], name="val_mae"))
    fig.show()

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=history.epoch, y=history.history["loss"], name="loss"))
    fig2.add_trace(go.Scatter(x=history.epoch, y=history.history["val_loss"], name="val_loss"))
    fig2.show()


draw_history(history)

In [ ]:
# model.load_weights("model3.keras")

df = pd.DataFrame({"value": labels, "prediction": model.predict(data, verbose=1).ravel()})

print(f"Mean absolute error: {round(mean_absolute_error(df['value'], df['prediction']), 2)}")
df